In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

def getData(filePath):
    data = np.genfromtxt(filePath, delimiter=',')
    x, y = np.array(data[:,0:-1], dtype=float), np.array(data[:,-1],dtype=int)
    y = y.reshape(1,len(y)).T
    return x,y

def splitInputOutput(data):
    x, y = np.array(data[:,0:-1], dtype=float), np.array(data[:,-1],dtype=int)
    y = y.reshape(1,len(y)).T
    return x,y

In [8]:
import tensorflow as tf


df = pd.read_csv('data/prepared/dataWithTemp.csv',sep=',',names=["Temp", "Nausea", "Lumbar", "Pushing","Micturition","Burning","BladderInflamation"]);
df["Temp"] = df.transform(lambda x: (x - np.mean(x)) / np.std(x))

data_X,data_y = splitInputOutput(df.as_matrix())

kf = KFold(n_splits=5,random_state=None, shuffle=True)

for train_index, test_index in kf.split(data_X):
    X_train, X_test = data_X[train_index], data_X[test_index]
    y_train, y_test = data_y[train_index], data_y[test_index]

    x = tf.placeholder(tf.float32, [None, 96])
    W = tf.Variable(tf.zeros([96, 2]))
    b = tf.Variable(tf.zeros([2]))
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    y_ = tf.placeholder(tf.float32, [None, 2])
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    sess.run(train_step, feed_dict={x: X_train, y_: y_train})
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: X_test, y_: y_test}))

ValueError: Cannot feed value of shape (96, 6) for Tensor 'Placeholder_7:0', which has shape '(?, 96)'